# Quality control and data cleaning|

In [21]:
import pandas as pd
import pandasgui as pg
def cleanCol(col):
    tmp = []
    for elem in col: 
        tmp.append(( str(elem).strip() ) )
    return tmp

hackathon_curated = pd.read_csv('../data/assignment.csv', dtype={"Pubmed_id":"object"})
previously_curated = pd.read_csv('../data/to_curate_vars_2nd_round.csv', dtype={"Pubmed_id":"object"} )

# clean th pubmed id 
previously_curated["Pubmed_id"] = previously_curated.Pubmed_id.str.strip()
hackathon_curated["Pubmed_id"] = hackathon_curated.Pubmed_id.str.strip()



In [22]:
# generate temp ids 

all_ids = previously_curated.Variant_ID+previously_curated.Pubmed_id
hack_ids = hackathon_curated.Variant_ID+hackathon_curated.Pubmed_id

previously_annotated_snps = []
for id in list(all_ids): 
    if id not in list(hack_ids):
        previously_annotated_snps.append(id)

# get the snps that were annotated from the original doc 
bool_filter_list = []
for index, snp in previously_curated.iterrows() :
    combined_id = snp["Variant_ID"]+snp["Pubmed_id"]
    if combined_id in previously_annotated_snps:
        bool_filter_list.append(True)
    else: 
        bool_filter_list.append(False)

previously_curated = previously_curated[bool_filter_list]


# Filter raws to be removed and merge dataframes
The following code generates the concatenated dataframe of the raw data curated in the second round and outputs it to a csv file.

In [23]:
hackathon_curated = hackathon_curated[hackathon_curated["Comments"] != "REMOVE"]
hackathon_curated = hackathon_curated[ hackathon_curated.columns[0:10] ]
previously_curated= previously_curated[previously_curated.columns[0:10]]

curated_data = pd.concat([previously_curated, hackathon_curated])
curated_data.to_csv("../data/AGMP_curated_data_raw.csv", index=False)

# Quality control

In [105]:
curated_data.isna().sum()

Variant_ID                  0
Pubmed_id                   1
phenotype                   0
origin_of_participants    155
region                    162
Ethnicity                 571
mixed_population          341
p-value                   508
dbSNP Gene                192
Notes                     847
dtype: int64

First all the missing values are replaced by NS

In [107]:
curated_data.fillna("NS", inplace=True)
curated_data.isna().sum()

Variant_ID                0
Pubmed_id                 0
phenotype                 0
origin_of_participants    0
region                    0
Ethnicity                 0
mixed_population          0
p-value                   0
dbSNP Gene                0
Notes                     0
dtype: int64

## The region field

In [99]:
import matplotlib.pylab as plt 
import seaborn as sns

#sns.countplot(curated_data.region, orient="h")
categories = curated_data.groupby('region').count()
#pg.show(categories)

In [98]:
curated_data.replace(['60 Caucasian-American (CA), 60 AfricanAmerican (AA), 60 Han Chinese-American (HCA) and 60 Mexican-American (MA)', 
                     '64 White (Non-Hispanic),194  African-American (Non-Hispanic),130 Hispanic,10 Asians and Pacific Islanders and others ',
                     'African American', 'African American and Hispanic American', 'African American/Afro-Caribbean, southeastern United States', 
                     '730 of European American and 143 of African American descent', 'African-American', 'Afro-Caribbean', 
                      'European-Americans and African-Americans', 'GERA (81% self-identified non-Hispanic white (NHW) participants, and 19% self-identified ethnic minorities (7% Asian, 3.5% African American, 7% Latinos, and 1.5% Other))',
                     'PEAR: 461 Caucasian and 298 African American',
       'PEAR: white (60.6%),African-American (39.4%)',
       'PEAR: white (60.6%),African-American (39.4%).INVEST:White,Hispanic,African-American', 
                      'self-identified African-American (USA)', 'African–American', 'African American/Afro-Caribbean', 'race self-reported'],
                    'African-American/Afro-Caribbean', inplace=True)

curated_data.replace(['Eastern Mediterranean and North Africa', 'Morrocon and Palistaneans', 
                      'Near Eastern/ North Africa', 'Tunisian ', 'Tunusia',
                      'south Europe and North Africa', 'North Africa,', 'Northen Africa', 'Pooled population'], "North Africa", inplace=True)

curated_data.replace(['Africa ', 'African Populations',  'African, Asian, Caucasian, Hispanic',
                     ], "Africa", inplace=True)

curated_data.replace(['Eastern Africa','East Africa & non-African', 'Malawi', 'Zimbabwe', 'Tanzania'],
                     'East Africa', inplace=True)

curated_data.replace(['African-Americans and Native Africans','South Africa', 'South Africa and Australia'
                     ], "Southern Africa", inplace=True)

curated_data.replace(['Eastern Africa / Southeastern Africa /Western Africa',
                     ], 'East Africa,Southern Africa,Western Africa', inplace=True)

curated_data.replace(['Nigeria', 'Nigerians', 'Western Africa'], 
                     'Western Africa', inplace=True)

curated_data.replace(['Sub-Saharan Africa/African American/Afro-Caribbean'], 
                     'Sub-Saharan Africa,African-American/Afro-Caribbean', inplace=True)


curated_data.replace(["Sub-Saharan African"], 
                     'Sub-Saharan Africa', inplace=True)

# any variants belonging to these regions are dropped 
curated_data.drop(curated_data[curated_data.region.isin(["East Asian", 'Europe', 'Isreal', 'Mexico city', 
                                                        'Mixed Population (Lattino and whites)', 'Near Eastern', 'New Zeland',
                                                        'Non-Hispanic whites', 'Not_African', 'South America', 
                                                        'predominantly central European', 'Central/South Asian'])].index, inplace=True)

categories = curated_data.groupby('region').count()
categories

In [100]:
categories.index

Index(['Africa', 'African-American/Afro-Caribbean', 'Central Africa',
       'East Africa', 'East Africa, Southern Africa',
       'East Africa,Southern Africa,Western Africa', 'North Africa',
       'Southern Africa', 'Southern Africa,East Africa', 'Sub-Saharan Africa',
       'Sub-Saharan Africa,African-American/Afro-Caribbean', 'West Africa',
       'Western Africa'],
      dtype='object', name='region')

In [104]:
categories = curated_data.groupby('region').count()
categories

,Variant_ID,Pubmed_id,phenotype,origin_of_participants,Ethnicity,mixed_population,p-value,dbSNP Gene,Notes
region,,,,,,,,,
Africa,36,36,36,35,31,35,33,33,30
African-American/Afro-Caribbean,313,313,313,302,190,284,283,309,154
Central Africa,56,55,56,56,56,55,49,56,51
East Africa,79,79,79,63,56,72,53,79,45
"East Africa, Southern Africa",5,5,5,5,0,5,0,5,5
"East Africa,Southern Africa,Western Africa",19,19,19,19,19,19,19,19,7
North Africa,940,940,940,928,733,811,722,892,556
Southern Africa,216,216,216,216,203,199,192,213,159
"Southern Africa,East Africa",7,7,7,7,7,7,7,7,7


## Origin of participants

In [111]:
categories = curated_data.groupby('origin_of_participants').count()
pg.show(categories)

# transform and Output cleaned data

In [113]:


curated_data.to_csv("../data/row_curateddata.csv")

In [114]:
pg.show(curated_data)